In [67]:
from deep_translator import GoogleTranslator
import json
import glob
import re
import os
import pandas as pd



In [56]:
def split_string_into_chunks(text, max_chunk_size=500):
    # Use regular expressions to split the text into sentences
    # Assumes sentences end with ".", "!", or "?", followed by a space or newline
    sentences = re.split(r'(?<=[.!?])\s+', text)
 
    # Initialize variables
    current_chunk = ""
    chunks = []
 
    # Iterate through sentences and add them to the current chunk
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            # If adding the sentence would exceed the chunk size, start a new chunk
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
 
    # Add the last chunk, if any
    if current_chunk:
        while len(current_chunk) > 50:
            chunks.append(current_chunk.strip())
 
    return chunks

In [68]:
def translate(path):
    print(rf'{path}/*.csv')
    for file in glob.glob(rf'{path}/*.csv'):
        print(file)
        df = pd.read_csv(file, index_col="Unnamed: 0", engine="python")
        
        for ind, _ in df.iterrows():
            chunks = split_string_into_chunks(df.at[ind, "text"])
            df.at[ind, "text"] = ' '.join(GoogleTranslator(source='auto', target='english').translate_batch(chunks))
        

translate("data")

data/*.csv
data/articles.csv


NotValidLength: ҚАМ" БК" ЖШС мұнай кәсіпшілігі нысандарын кеңейту – 2021 ж (түзетулер)" жұмыс жобасы https://www.youtube.com/watch?v=B8UAQC8XCQA&feature=youtu.be Орналастыру күні Хабарландыру 11.01.2022 ж "ҚуатАмлонМұнай" БК" ЖШС ҚР Экологиялық кодексінің 96-бабына сәйкес 2022 жылғы 14 ақпан күні сағат 11.00-де Қызылорда облысы, Сырдария ауданы, Тереңөзек селосы, Абай көшесі 31 мекенжайы бойынша "ҚАМ" БК" ЖШС мұнай кәсіпшілігі нысандарын кеңейту – 2021 ж (түзетулер)" жұмыс жобасының "Қоршаған ортаны қорғау бөлімі" жобасы бойынша ашық жиналыс түрінде қоғамдық тыңдаулар өтетіндігін хабарлайды. --> Text length need to be between 0 and 500 characters